#### 기존 특징으로 새로운 특징 만들기

신경망에서 가장 많이 사용되는 연산은 특징의 가중합(weighted sum)을 구하는 연산이다. 

이 가중합은 특정한 특징의 비중을 늘리거나 줄이는 방법으로 기존 특징을 조합해 새로운 특징을 만든다.

가중합을 간략히 표현하면 각 특징에 대한 일련의 가중치를 모은 벡터와 데이터 점 벡터의 점곱이다.

In [2]:
from numpy import ndarray
import numpy as np
def matmul_forward(x: ndarray,
                   W: ndarray) -> ndarray:
  """
  순방향 계산을 행렬곱으로 계산
  """
  assert X.shape[1] == W.shape[0], \
  f"""
  행렬곱을 계산하려면 첫 번째 배열의 열의 개수와
  두 번째 배열의 행의 개수가 일치해야 한다.
  그러나 지금은 첫 번째 배열의 열의 개수가 {X.shape[1]}이고
  두 번째 배열의 행의 개수가 {W.shape[0]}이다.
  """

  # 행렬곱 연산
  N = np.dot(X, W)
  
  return N

#### 여러 개의 벡터 입력을 갖는 함수의 도함수

In [3]:
def matmul_backward_first(X: ndarray,
                          W: ndarray) -> ndarray:
  """
  첫 번째 인자에 대한 행렬곱의 역방향 계산 수행
  """

  # 역방향 계산
  dNdX = np.transpose(W, (1, 0))

  return dNdX

#### 벡터 함수와 도함수

In [5]:
from typing import Callable

# ndarray를 인자로 받고 ndarray를 반환하는 함수
Array_Function = Callable[[ndarray], ndarray]

# Chain은 함수의 리스트다.
Chain = List[Array_Function]

In [6]:
def matrix_forward_extra(X: ndarray,
                         W: ndarray,
                         sigma: Array_Function) -> ndarray:
  """
  행렬곱이 포함된 함수와 또 다른 함수의 합성함수에 대한 순방향 계산을 수행
  """
  assert X.shape[1] == W.shape[0]

  # 행렬곱
  N = np.dot(X,W)

  # 행렬곱의 출력을 함수 sigma의 입력값으로 전달
  S = sigma(N)

  return S

#### 역방향 계산

In [16]:
from typing import Callable

def deriv(func: Callable[[ndarray], ndarray],
          input_: ndarray,
          delta: float = 0.001) -> ndarray:
          """
          배열 input의 각 요소에 대해 함수 func의 도함숫값 계산
          """
          return (func(input_ + delta) - func(input_ - delta)) / (2 * delta)

In [8]:
def matrix_function_backward_1(X: ndarray,
                               W: ndarray,
                               sigma: Array_Function) -> ndarray:
  """
  첫 번쨰 요소에 대한 행렬함수의 도함수 계산
  """
  assert X.shape[1] == W.shape[0]

  # 행렬곱
  N = np.dot(X, W)

  # 행렬곱의 출력을 함수 sigma의 입력값으로 전달
  S = sigma(N)

  # 역방향 계산
  dSdN = deriv(sigma, N)

  # dNdX
  dNdX = np.transpose(W, (1, 0))

  # 계산한 값을 모두 곱함. 여기서는 dNdX의 모양이 1*1이므로 순서는 무관한
  return np.dot(dSdN, dNdX)

#### 두 개의 2차원 행렬을 입력받는 계산 그래프

In [9]:
def matrix_function_forward_sum(X: ndarray,
                                W: ndarray,
                                sigma: Array_Function) -> float:
  """
  두 개의 ndarray X와 W를 입력받으며 sigma 함수를 포함하는 합성함수의 순방향 계산
  """
  assert X.shape[1] == W.shape[0]

  # 행렬곱
  N = np.dot(X, W)

  # 행렬곱 계산 결과를 sigma에 전달
  S = sigma(N)

  # 행렬 요소의 합을 구함
  L = np.sum(S)

  return L

#### 역방향 계산

In [14]:
def matrix_function_backward_sum_1(X: ndarray,
                                   W: ndarray,
                                   sigma: Array_Function) -> ndarray:
  """
  행렬곱과 요소의 합 연산이 포함된 함수의
  첫 번째 인자 행렬에 대한 도함수를 계산하는 과정 구현
  """
  assert X.shape[1] == W.shape[0]

  # 행렬곱
  N = np.dot(X, W)

  # 행렬곱 계산 결과를 sigma에 전달
  S = sigma(N)

  # 행렬 요소의 합을 구함
  L = np.sum(S)

  # 메모: 수식에서 도함수를 가리키는 부분을 여기서는 계산된 값으로 다룬다.

  # dLdS - 모든 요솟값이 1인 행렬
  dLdS = np.ones_like(S)

  # dSdN
  dSdN = deriv(sigma, N)

  # dLdN
  dLdN = dLdS * dSdN

  # dNdX
  dNdX = np.transpose(W, (1, 0))

  # dLdX
  dLdX = np.dot(dSdN, dNdX)

  return dLdX

In [12]:
def sigmoid(x: ndarray) -> ndarray:
  """
  입력으로 받은 ndarray의 각 요소에 대한 sigmoid 함숫값을 계산한다.
  """
  return 1 / (1 + np.exp(-x))

In [17]:
# 검증
np.random.seed(190204)

X = np.random.randn(3, 3)
W = np.random.randn(3, 2)

print('X:')
print(X)

print('L:')
print(round(matrix_function_forward_sum(X, W, sigmoid), 4))
print()
print('dLdX:')
print(matrix_function_backward_sum_1(X, W, sigmoid))

X:
[[-1.57752816 -0.6664228   0.63910406]
 [-0.56152218  0.73729959 -1.42307821]
 [-1.44348429 -0.39128029  0.1539322 ]]
L:
2.3755

dLdX:
[[ 0.2488887  -0.37478057  0.01121962]
 [ 0.12604152 -0.27807404 -0.13945837]
 [ 0.22992798 -0.36623443 -0.02252592]]


In [22]:
# dLdX는 X에 대한 L의 기울기
# 계산 결과 검증 - x₁₁을 0.001 증가시키면 L도 0.001 x 0.2489 증가 해야함.

X1 = X.copy()
X1[0, 0] += 0.001

print(round(
    (matrix_function_forward_sum(X1, W, sigmoid) - matrix_function_forward_sum(X, W, sigmoid)) / 0.001, 4))

0.2489
